# Problem 3


Problem 3 (50 pt).
Find ways to solve this SOCP optimization problem. Then use the solution w∗, b∗from this  optimization to form aclassifier.

USE cvxpy

In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from time import time


(CVXPY) May 21 07:24:57 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: 找不到指定的模块。')


In [2]:
#封装MPM类
class MPM():
    def __init__(self) -> None:
        pass
    def optimize(self):
        #分两类
        x_p = self.X[self.y == 1]
        x_n = self.X[self.y == 0]
        #分别计算协方差矩阵
        cov_p = np.cov(x_p.T)
        cov_n = np.cov(x_n.T)
        #均值
        mu_p = np.mean(x_p,axis=0)
        mu_n = np.mean(x_n,axis=0)
        dim = mu_n.shape[0]
        #1/2的协方差矩阵
        Q_p = np.linalg.cholesky(cov_p)
        Q_n = np.linalg.cholesky(cov_n)
        #开始优化
        #定义优化的变量
        w = cp.Variable(dim)
        equa = cp.norm(Q_n.T@w,2)+cp.norm(Q_p.T@w,2)
        prob = cp.Problem(cp.Minimize(equa),[(mu_p-mu_n).T @ w ==1])
        prob.solve()
        #最优解
        w_opt = w.value
        k_opt = 1 / (np.sqrt(w_opt.T @ cov_p @ w_opt) + np.sqrt(w_opt.T @ cov_n @ w_opt))
        b_opt = w_opt@mu_p - k_opt*np.sqrt(w_opt.T @ cov_n @ w_opt)
        #计算好最优后定义此分类器参数
        self.w = w_opt
        self.b = b_opt
        self.e = 1 / (k_opt**2 + 1)

    def fit(self,X,y):
        self.X = X
        self.y = y
        self.optimize()
    #根据得到的分类器来预测
    def predict(self,X):
        y = (self.w) @ (X.T) - self.b
        y[y >= 0] = 1
        y[y < 0] = 0
        return y


Compare its performance with the original LDA, Logistic Regression, and SVM on the datasets
we provide (including breastcancer, pima, sonar). Each time you should randomly partition it
into 90% training set and 10% test set, and repeat that procedure 10 times to get the average
test accuracy. Also report the average MPM’s guaranteed error e in line (1). You should achieve
comparable results as in paper[1].

首先读取数据

（1）breastcancer

In [3]:
def del_question(df):# 直接删掉含有？的行
    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df = df.dropna(subset=[col],axis=0)
    return df
def read_breastcancer(path ='./data/breast-cancer-wisconsin.txt' ):
    data_breastcancer = pd.read_csv(path,sep='\t',header=None)
    # 直接删掉含有？的行
    data_breastcancer = del_question(data_breastcancer)
    benigndata = data_breastcancer[data_breastcancer[10]==0].values
    maligndata = data_breastcancer[data_breastcancer[10]==1].values
    # print(benigndata)
    fulldata = np.append(benigndata,maligndata,axis=0)
    # print(fulldata.shape)
    X = fulldata[:,1:10]
    y = fulldata[:,10]
    return X,y
# X,y = read_breastcancer()
# X.shape
# y.shape

(2) diabetes

In [4]:
def read_diabetes(path = './data/diabetes.csv'):
    data_diabetes = pd.read_csv(path)
    # display(data_diabetes)
    class1 = data_diabetes[data_diabetes['Outcome']==1].values
    class2 = data_diabetes[data_diabetes['Outcome']==0].values
    fulldata = np.append(class1,class2,axis=0)
    X = fulldata[:,0:8]
    y = fulldata[:,8]
    return X,y
# X, y=read_diabetes()
# y.shape

(3) sonar

In [5]:
def read_sonar(path = './data/sonar_csv.csv'):
    data_sonar = pd.read_csv(path)
    data_sonar['Class'] = data_sonar['Class'].replace({'Rock':1,'Mine':0})
    data_sonar=data_sonar.values
    X = data_sonar[:,0:60]
    y = data_sonar[:,60]
    return X,y
# X,y = read_sonar()
# X.shape

开始训练

In [6]:

def train(X, y, model_MPM:MPM, model_LDA:LinearDiscriminantAnalysis, model_LR:LogisticRegression, model_SVM:SVC, name:str):
    MPM_err = 0
    MPM_acc = 0
    LDA_acc = 0
    LR_acc = 0
    SVM_acc = 0
    MPM_t = 0
    LDA_t = 0
    LR_t = 0
    SVM_t = 0
    for i in range(0, 10):
        X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.1,random_state = i + 1,shuffle = True)

        t1 = time()
        model_MPM.fit(X_train, y_train)
        t2 = time()
        model_LDA.fit(X_train, y_train)
        t3 = time()
        model_LR.fit(X_train, y_train)
        t4 = time()
        model_SVM.fit(X_train, y_train)
        t5 = time()

        y_MPM = model_MPM.predict(X_test)
        y_LDA = model_LDA.predict(X_test)
        y_LR = model_LR.predict(X_test)
        y_SVM = model_SVM.predict(X_test)

        MPM_acc += np.sum(y_MPM == y_test) / y_test.shape[0]
        LDA_acc += np.sum(y_LDA == y_test) / y_test.shape[0]
        LR_acc += np.sum(y_LR == y_test) / y_test.shape[0]
        SVM_acc += np.sum(y_SVM == y_test) / y_test.shape[0]

        MPM_t = MPM_t + (t2 - t1)
        LDA_t = LDA_t + (t3 - t2)
        LR_t = LR_t + (t4 - t3)
        SVM_t = SVM_t + (t5 - t4)

        MPM_err += model_MPM.e
    MPM_acc /= 10
    MPM_err /= 10
    LDA_acc /= 10
    LR_acc /= 10
    SVM_acc /= 10
    MPM_t /= 10
    LDA_t /= 10
    LR_t /= 10
    SVM_t /= 10
    print(name)
    print('LDA———— Accuracy: ', LDA_acc, 'Time: ', LDA_t)
    print('LGS———— Accuracy: ', LR_acc, 'Time: ', LR_t)
    print('SVM———— Accuracy: ', SVM_acc, 'Time: ', SVM_t)
    print('MPM————Accuracy: ', MPM_acc, 'Time: ', MPM_t)
    print('MPM guaranteed error: ', MPM_err)


运行

In [7]:
X_breastcancer,y_breastcancer= read_breastcancer()
X_diabetes,y_diabetes= read_diabetes()
X_sonar,y_sonar = read_sonar()
model_MPM =MPM()
model_LDA = LinearDiscriminantAnalysis()
model_LR = LogisticRegression()
model_SVM = SVC()
train(X_breastcancer,y_breastcancer,model_MPM,model_LDA,model_LR,model_SVM,'breastcancer')
train(X_diabetes,y_diabetes,model_MPM,model_LDA,model_LR,model_SVM,'diabetes')
train(X_sonar,y_sonar,model_MPM,model_LDA,model_LR,model_SVM,'sonar')



breastcancer
LDA———— Accuracy:  0.9594202898550724 Time:  0.002652406692504883
LGS———— Accuracy:  0.9681159420289855 Time:  0.012859487533569336
SVM———— Accuracy:  0.9637681159420289 Time:  0.006972265243530273
MPM————Accuracy:  0.9405797101449276 Time:  0.013753199577331543
MPM guaranteed error:  0.15526610718499848


d:\Anaconda\envs\prml\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Anaconda\envs\prml\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

diabetes
LDA———— Accuracy:  0.7740259740259742 Time:  0.002436065673828125
LGS———— Accuracy:  0.7636363636363637 Time:  0.02573072910308838
SVM———— Accuracy:  0.7753246753246754 Time:  0.026667189598083497
MPM————Accuracy:  0.7597402597402598 Time:  0.01182544231414795
MPM guaranteed error:  0.677132391654075
sonar
LDA———— Accuracy:  0.738095238095238 Time:  0.004142093658447266
LGS———— Accuracy:  0.7571428571428571 Time:  0.00808563232421875
SVM———— Accuracy:  0.7952380952380953 Time:  0.003951764106750489
MPM————Accuracy:  0.7428571428571428 Time:  0.019269919395446776
MPM guaranteed error:  0.3536017233036554
